# Классификация мусора по комментариям (образование)

## 1. EDA

In [36]:
import re
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


In [37]:
df = pd.read_csv('data/data.csv', quotechar='"')

In [38]:
df.head()

,text,relevant
0,"Единственный в стране, нисколько не сомневаюсь...",1
1,Замечательная женщина и преподаватель!🌺,1
2,"Пересдача,курсовая ,удача 😁не все так плохо в...",1
3,Для потенциальных абитуриентов (студентов) пол...,1
4,"Рейтинг Вуза это конечно здорово, но 5 бюджетн...",1


Предобработка данных

In [39]:
def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext


def cleanPunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned


def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^а-я А-Я]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

In [40]:
text = 'text'

df[text] = df[text].str.lower()
df[text] = df[text].apply(cleanHtml)
df[text] = df[text].apply(cleanPunc)
df[text] = df[text].apply(keepAlpha)
df.head()

,text,relevant
0,единственный в стране нисколько не сомневаюсь ...,1
1,замечательная женщина и преподаватель,1
2,пересдача курсовая удача не все так плохо вро...,1
3,для потенциальных абитуриентов студентов получ...,1
4,рейтинг вуза это конечно здорово но бюджетны...,1


Удаление стопслов, стемминг

In [41]:
stop_words = set(stopwords.words('russian'))
stop_words.update(['ок','один','два','три','четыре','пять','шесть','семь','восемь','девять','десять','может','также'])

def removeStopWords(sentence):
    re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
    return re_stop_words.sub(" ", sentence)

df[text] = df[text].apply(removeStopWords)
df.head()

,text,relevant
0,единственный стране нисколько сомневаюсь р...,1
1,замечательная женщина преподаватель,1
2,пересдача курсовая удача плохо вроде бы,1
3,потенциальных абитуриентов студентов получить...,1
4,рейтинг вуза это здорово бюджетных мест п...,1


In [42]:
stemmer = SnowballStemmer('russian')
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

df[text] = df[text].apply(stemming)
df.head()

,text,relevant
0,единствен стран нискольк сомнева ректор универ...,1
1,замечательн женщин преподавател,1
2,пересдач курсов удач плох врод бы,1
3,потенциальн абитуриент студент получ информац ...,1
4,рейтинг вуз эт здоров бюджетн мест поступлен н...,1


In [43]:
train, test = train_test_split(df, test_size=0.25, random_state=0, shuffle=True)

In [44]:
train_text = train[text]
test_text = test[text]

y_train = train['relevant']
y_test = test['relevant']

In [45]:
train_text.shape, test_text.shape

((32664,), (10889,))

In [46]:
vectorizer = TfidfVectorizer(strip_accents='unicode')
vectorizer.fit(train_text)

TfidfVectorizer(strip_accents='unicode')

Векторизация текстов

In [47]:
X_train = vectorizer.transform(train_text)
X_test = vectorizer.transform(test_text)

In [17]:
def calculate_metrics(model_fitted, X_test):
    y_pred = model_fitted.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Вывод результатов
    print('Test accuracy is {}'.format(accuracy))
    print('Test precision is {}'.format(precision))
    print('Test recall is {}'.format(recall))
    print('Test f1 is {}'.format(f1))


Метод опорных векторов

In [18]:
classificatorSVC = SVC(kernel='linear')
classificatorSVC.fit(X_train, y_train)


SVC(kernel='linear')

In [19]:
# Вывод результатов
calculate_metrics(classificatorSVC, X_test)

Test accuracy is 0.7593902103039765
Test precision is 0.7928725456815344
Test recall is 0.8978280426680375
Test f1 is 0.8420925747348119


Дерево решений

In [20]:
classificatorDT = DecisionTreeClassifier(class_weight='balanced')
classificatorDT.fit(X_train, y_train)


DecisionTreeClassifier(class_weight='balanced')

In [21]:
# Вывод результатов
calculate_metrics(classificatorDT, X_test)


Test accuracy is 0.6845440352649462
Test precision is 0.8045123318385651
Test recall is 0.7378229019406246
Test f1 is 0.7697258161828786


Логит. Регрессия

In [22]:
classificatorLR = LogisticRegression(class_weight='balanced')
classificatorLR.fit(X_train, y_train)


LogisticRegression(class_weight='balanced')

In [23]:
# Вывод результатов
calculate_metrics(classificatorLR, X_test)

Test accuracy is 0.7252272935990449
Test precision is 0.8708378503949202
Test recall is 0.7226577560724843
Test f1 is 0.789858126141312


Градиентный бустинг (Адабуст)

In [24]:
classificatorADA = AdaBoostClassifier()
classificatorADA.fit(X_train, y_train)


d:\pet\comment-classifier\venv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier()

In [25]:
# Вывод результатов
calculate_metrics(classificatorADA, X_test)


Test accuracy is 0.7324823216089632
Test precision is 0.8008652657601978
Test recall is 0.8326693227091634
Test f1 is 0.816457690126646


Случайный лес

In [26]:
classificatorRF = RandomForestClassifier(class_weight='balanced')
classificatorRF.fit(X_train, y_train)


RandomForestClassifier(class_weight='balanced')

In [27]:
# Вывод результатов
calculate_metrics(classificatorRF, X_test)


Test accuracy is 0.7578290017448801
Test precision is 0.7894440693225299
Test recall is 0.901555070042411
Test f1 is 0.8417831643367133


Градиентный бустинг

In [28]:
classificatorGB = GradientBoostingClassifier()
classificatorGB.fit(X_train, y_train)


GradientBoostingClassifier()

In [29]:
# Вывод результатов
calculate_metrics(classificatorGB, X_test)


Test accuracy is 0.7209110111121315
Test precision is 0.720928065598211
Test recall is 0.9943451998457782
Test f1 is 0.8358450818343867


К - соседей

In [30]:
classificatorKN = KNeighborsClassifier()
classificatorKN.fit(X_train, y_train)


KNeighborsClassifier()

In [31]:
# Вывод результатов
calculate_metrics(classificatorKN, X_test)


Test accuracy is 0.33428230324180364
Test precision is 0.9182389937106918
Test recall is 0.07505462022876237
Test f1 is 0.13876678151360342


Проведем кросс-валидацию двух лучших моделей

In [48]:
classificatorSVC = SVC(kernel='linear')
classificatorRF = RandomForestClassifier(class_weight='balanced')


In [49]:
scores = cross_val_score(classificatorSVC, X_train, y_train, cv=5, scoring='f1')
print(f"F1 на каждом фолде: {scores}")
print(f"F1 точность: {scores.mean()}")

F1 на каждом фолде: [0.84520342 0.83978011 0.84814667 0.8428643  0.84306826]
F1 точность: 0.8438125511054396


In [50]:
scores = cross_val_score(classificatorRF, X_train, y_train, cv=5, scoring='f1')
print(f"F1 на каждом фолде: {scores}")
print(f"F1 точность: {scores.mean()}")

F1 на каждом фолде: [0.8420004  0.84000399 0.84310996 0.84477967 0.84275478]
F1 точность: 0.8425297609326714


В итоге наилучшей моделью (из классических) для прогнозирования релевантности текстов стала модель классификатора на основе метода опорных векторов. Она имеет наивысший f1, а также высокий recall около 0.9, что отлично подходит для нашей задачи.


Эту модель можно улучшить, путем подбора гиперпараметров (например методом поиска по сетке). Однако за неимением вычислительных мощностей остановимся на базовом варианте модели.